In [1]:
from robotic import ry
import numpy as np
import time

In [2]:
ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
#ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 0., 'physx/motorKd': 0.})
ry.params_add({'botsim/engine': 'physx'}) #makes a big difference!
ry.params_add({'physx/multibody': True}) #makes a big difference!
ry.params_print()

-- ry.cpp:operator():99(0) python,
botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody: 1


In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

In [4]:
bot = ry.BotOp(C, False)

-- kin_physx.cpp:PhysXInterface:768(0) starting PhysX engine ...
-- kin_physx.cpp:addGround:238(0) ... done starting PhysX engine
-- kin_physx.cpp:addGround:239(0) creating Configuration within PhysX ...
-- kin_physx.cpp:addLink:254(0) adding link 'world' as static with 1 shapes
 table
-- kin_physx.cpp:addMultiBody:466(0) adding multibody with base 'l_panda_base' with the following links ...
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_base' as kinematic with 1 shapes
 l_panda_link0_0
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint1' as dynamic with 1 shapes
 l_panda_link1_0
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint2' as dynamic with 2 shapes
 l_panda_link2_0 bellybutton
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint3' as dynamic with 1 shapes
 l_panda_link3_0
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint4' as dynamic with 1 shapes
 l_panda_link4_0
-- 

In [7]:
del bot
del C

-- bot.cpp:~BotOp:106(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


In [5]:
qHome = bot.get_qHome()
q0 = qHome.copy()
q1 = q0.copy()
q1[1] = q1[1] + .2
print(q0, q1)

[ 0.  -0.5  0.  -2.   0.   2.  -0.5] [ 0.  -0.3  0.  -2.   0.   2.  -0.5]


In [6]:
bot.moveTo(q1)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [7]:
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q0)
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q1)
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q0)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

timeToEnd: -218.48299670517036
timeToEnd: 1.095718942064991
timeToEnd: 2.191437884129982


In [8]:
#this reference frame only appears in your workspace C - not the simulation!
target = C.addFrame('target', 'table')
target.setShape(ry.ST.marker, [.1])
target.setRelativePosition([0., .3, .3])
pos = target.getPosition()
cen = pos.copy()
C.view()

def IK(C, pos):
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 0, False) #one phase one time slice problem, with 'delta_t=1', order=0
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) #cost: close to 'current state'
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'target'], ry.OT.eq, [1e1]) #constraint: gripper position

    ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()

    return [komo.getPath()[0], ret]

In [9]:
for t in range(20):
    time.sleep(.1)
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)

    q_target, ret = IK(C, pos)
    print(ret)
    C.setJointState(q_target)
    C.view()

{ time: 0.001605, evals: 6, done: 1, feasible: 1, sos: 0.00574408, f: 0, ineq: 0, eq: 0.000719999 }
{ time: 0.000719, evals: 4, done: 1, feasible: 1, sos: 0.00362931, f: 0, ineq: 0, eq: 0.000288095 }
{ time: 0.000348, evals: 2, done: 1, feasible: 1, sos: 0.00346734, f: 0, ineq: 0, eq: 0.00492158 }
{ time: 0.00016, evals: 3, done: 1, feasible: 1, sos: 0.00261449, f: 0, ineq: 0, eq: 0.00589666 }
{ time: 0.000731, evals: 4, done: 1, feasible: 1, sos: 0.00232959, f: 0, ineq: 0, eq: 0.000239577 }
{ time: 0.000606, evals: 3, done: 1, feasible: 1, sos: 0.00214299, f: 0, ineq: 0, eq: 0.00104024 }
{ time: 0.000425, evals: 4, done: 1, feasible: 1, sos: 0.00287394, f: 0, ineq: 0, eq: 0.000349643 }
{ time: 0.000731, evals: 4, done: 1, feasible: 1, sos: 0.00355377, f: 0, ineq: 0, eq: 0.000318178 }
{ time: 0.000484, evals: 3, done: 1, feasible: 1, sos: 0.00284609, f: 0, ineq: 0, eq: 0.00220867 }
{ time: 0.000252, evals: 3, done: 1, feasible: 1, sos: 0.00237413, f: 0, ineq: 0, eq: 0.00276411 }
{ time

In [68]:
for t in range(100):
    bot.sync(C, .1) #keep the workspace C sync'ed to real/sim, and idle .1 sec
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)

    q_target, ret = IK(C, pos)
    bot.moveTo(q_target, timeCost=5., overwrite=True)

In [67]:
bot.sync(C, .1) #keep the workspace C sync'ed to real/sim, and idle .1 sec
pos = cen + .9 * (pos-cen) + 0.1 * np.random.randn(3)
target.setPosition(pos)

q_target, ret = IK(C, pos)
#bot.moveTo(q_target, timeCost=0.1, overwrite=True)
bot.moveTo(q_target, timeCost=100, overwrite=True)

In [70]:
for t in range(5):
    bot.moveTo(q1)
    bot.wait(C) #same as 'loop sync til keypressed or endOfTime', but also raises user window
    if bot.getKeyPressed()==ord('q'):
        break;

    bot.moveTo(q0)
    bot.wait(C)
    if bot.getKeyPressed()==ord('q'):
        break;

bot.home(C)

In [73]:
bot.gripperMove(ry._left, width=.02)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.gripperMove(ry._left, width=.075)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

-- simulation.cpp:moveGripper:333(1) initiating opening gripper l_panda_joint7 (without releasing obj) width:0.02 speed:0.2
-- simulation.cpp:modConfiguration:812(1) terminating opening gripper l_panda_joint7 at width 0.01998
-- simulation.cpp:moveGripper:333(1) initiating opening gripper l_panda_joint7 (without releasing obj) width:0.075 speed:0.2
-- simulation.cpp:modConfiguration:812(1) terminating opening gripper l_panda_joint7 at width 0.04


In [75]:
del bot
del C

-- bot.cpp:~BotOp:106(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


In [74]:
ry.params_print()

-- ry.cpp:operator():99(0) python,
botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody: 1,
bot/useGripper,
bot/useRobotiq,
bot/useArm: both,
botsim/hyperSpeed: 1,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBody,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
bot/useOptitrack!,
bot/useAudio!,
bot/raiseWindow!,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/useFCL,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopIters: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
opt/muInit: 1,
opt/muInc: 5,
opt/muMax: 10000,
opt/muLBInit: 0.1,
opt/muLBDec: 0.2,
opt/maxL